In [ ]:
!pip install -q bitsandbytes accelerate transformers datasets peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset
import torch

MODEL_ID = "capston-team-5/finma-7b-4bit-quantized"
DATA_URL = "https://raw.githubusercontent.com/Stock-XAI/Data/refs/heads/main/regression/kospi_10_days_reg_output_.jsonl"

In [ ]:
dataset = load_dataset("json", data_files=DATA_URL, split="train")

# 간단한 포맷 확인
print(dataset[0])

# 텍스트 필드를 'prompt'로 추정 (예시로 "instruction", "output" 등의 필드를 하나로 합침)
def format_prompt(example):
    return {
        "text": f"### Input:\n{example['instruction']}\n\n### Output:\n{example['output']}"
    }

dataset = dataset.map(format_prompt)

Generating train split: 0 examples [00:00, ? examples/s]

{'instruction': 'Using the context below, estimate the rate of change in the closing price of 삼성전자 on 2022-01-17.\nReturn the expected value of change as a decimal.\n\nContext: date, open, high, low, close, volume, change.\n2022-01-03, 79400, 79800, 78200, 78600, 13502112, 0.001272\n2022-01-04, 78800, 79200, 78300, 78700, 12427416, -0.016518\n2022-01-05, 78800, 79000, 76400, 77400, 25470640, -0.00646\n2022-01-06, 76700, 77600, 76600, 76900, 12931954, 0.018205\n2022-01-07, 78100, 78400, 77400, 78300, 15163757, -0.003831\n2022-01-10, 78100, 78100, 77100, 78000, 9947422, 0.011538\n2022-01-11, 78400, 79000, 78000, 78900, 13221123, 0.0\n2022-01-12, 79500, 79600, 78600, 78900, 11000502, -0.012674\n2022-01-13, 79300, 79300, 77900, 77900, 13889401, -0.007702\n2022-01-14, 77700, 78100, 77100, 77300, 10096725, 0.002587\nAnswer:', 'output': -0.006452}


Map:   0%|          | 0/3648 [00:00<?, ? examples/s]

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # Padding 설정

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

model = prepare_model_for_kbit_training(model)

tokenizer_config.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:609: UserWarning: `pad_token_id` should be positive but got -1. This will cause errors when batch generating, if there is padding. Please set `pad_token_id` explicitly as `model.generation_config.pad_token_id=PAD_TOKEN_ID` to avoid errors in generation
  warnings.warn(


generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 79,953,920 || all params: 6,818,369,536 || trainable%: 1.1726


In [ ]:
def tokenize_fn(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_fn, remove_columns=["text"])

from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

Map:   0%|          | 0/3648 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./finma-7b-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    logging_steps=20,
    num_train_epochs=5,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    fp16=True,
    gradient_checkpointing=True,
    save_strategy="epoch",
    save_total_limit=2,
    save_safetensors=True,
    report_to="none",
    optim="paged_adamw_8bit",
    torch_compile=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

<ipython-input-8-80ed97956a3a>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


Step,Training Loss
20,1.199100
40,0.733000
60,0.633000
80,0.597000
100,0.577600
120,0.579600
140,0.566600
160,0.555100
180,0.567700
200,0.550200


TrainOutput(global_step=1140, training_loss=0.5174015446713096, metrics={'train_runtime': 6469.7662, 'train_samples_per_second': 2.819, 'train_steps_per_second': 0.176, 'total_flos': 3.747112152779981e+17, 'train_loss': 0.5174015446713096, 'epoch': 5.0})

In [ ]:
model.save_pretrained("./finma-7b-lora")
tokenizer.save_pretrained("./finma-7b-lora")

('./finma-7b-lora/tokenizer_config.json',
 './finma-7b-lora/special_tokens_map.json',
 './finma-7b-lora/tokenizer.model',
 './finma-7b-lora/added_tokens.json',
 './finma-7b-lora/tokenizer.json')

In [ ]:
!pip install -q finance-datareader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00


In [ ]:
import FinanceDataReader as fdr
from datetime import datetime, timedelta

# 날짜 설정
today = datetime.today()
start_date = (today - timedelta(days=14)).strftime('%Y-%m-%d')

# 삼성전자 데이터 로딩
# df = fdr.DataReader("005930", start=start_date)
df = fdr.DataReader("000660", start=start_date)

# 수익률 계산 (전일 대비 종가 변화율)
df['Change'] = df['Close'].pct_change().fillna(0) * 100

# 최근 10일 데이터만 추출
last_10 = df.tail(10)

# context 문자열 생성
context = "\n".join([
    f"{idx.strftime('%Y-%m-%d')}, {int(row['Open'])}, {int(row['High'])}, {int(row['Low'])}, "
    f"{int(row['Close'])}, {int(row['Volume'])}, {row['Change']:.2f}"
    for idx, row in last_10.iterrows()
])

In [ ]:
prompt = (
    "Using the context below, estimate the rate of change in the closing price of SK하이닉스 on 2025-05-24.\n"
    "Return the expected value of change as a decimal.\n\n"
    "Context: date, open, high, low, close, volume, change.\n"
    f"{context}\n\nAnswer:"
)
print(prompt)

Using the context below, estimate the rate of change in the closing price of SK하이닉스 on 2025-05-24.
Return the expected value of change as a decimal.

Context: date, open, high, low, close, volume, change.
2025-05-12, 193600, 195200, 193500, 195000, 1624884, 2.58
2025-05-13, 200500, 201500, 197300, 198500, 3385009, 1.79
2025-05-14, 207000, 207000, 204000, 206000, 3333511, 3.78
2025-05-15, 207500, 208000, 199500, 200500, 3178957, -2.67
2025-05-16, 206000, 206750, 203500, 204500, 2419507, 2.00
2025-05-19, 203000, 203000, 197500, 199400, 2482673, -2.49
2025-05-20, 202500, 208000, 201500, 202000, 2820546, 1.30
2025-05-21, 204500, 205000, 199900, 200500, 1721760, -0.74
2025-05-22, 199900, 201500, 196500, 196900, 1988587, -1.80
2025-05-23, 199900, 201500, 198700, 199700, 1153238, 1.42

Answer:


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

base_model_path = "capston-team-5/finma-7b-4bit-quantized"  # 4bit 양자화 기반 모델
lora_model_path = "./finma-7b-lora"  # fine-tuned LoRA 모델 경로

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# tokenizer & base model
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    quantization_config=bnb_config,
    device_map="auto"
)

# LoRA 적용
model = PeftModel.from_pretrained(model, lora_model_path)
model.eval()

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id
    )

generated = tokenizer.decode(output[0], skip_special_tokens=True)
print("\n===== Model Output =====\n")
print(generated.replace(prompt, "").strip())


===== Model Output =====

-0.013746
2025-05-24, 199800, 204000, 199800, 1971


In [ ]:
from huggingface_hub import HfApi, HfFolder
from transformers import AutoTokenizer
from peft import PeftModel

# 업로드할 디렉토리 (LoRA 가중치가 저장된 경로)
lora_output_dir = "./finma-7b-lora"

# Hugging Face에 생성할 모델 이름
repo_name = "finma-7b-lora-regression-v1"  # 원하는 이름으로 변경 가능
hf_username = "capston-team-5"     # 본인의 HF 유저 이름으로 변경

# ✅ 토크나이저 저장 (선택적)
tokenizer.save_pretrained(lora_output_dir)

# ✅ 허깅페이스에 업로드
api = HfApi()

# 리포지토리 생성 (이미 존재하면 생략 가능)
api.create_repo(
    repo_id=f"{hf_username}/{repo_name}",
    private=True  # public으로 만들고 싶다면 False로 설정
)

# 업로드
from huggingface_hub import upload_folder

upload_folder(
    repo_id=f"{hf_username}/{repo_name}",
    folder_path=lora_output_dir,
    path_in_repo=".",  # 루트에 업로드
    commit_message="Upload LoRA 4bit adapter and tokenizer",
    # ignore_patterns=["*.ipynb", "*.pt", "*.bin"],  # 불필요한 파일 제외
)

adapter_model.safetensors:   0%|          | 0.00/320M [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/163M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/320M [00:00<?, ?B/s]

Upload 16 LFS files:   0%|          | 0/16 [00:00<?, ?it/s]

...

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/capston-team-5/finma-7b-lora-regression-v1/commit/d5199743e2ac34c19c6d55652dfc8f5be467c1d4', commit_message='Upload LoRA 4bit adapter and tokenizer', commit_description='', oid='d5199743e2ac34c19c6d55652dfc8f5be467c1d4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/capston-team-5/finma-7b-lora-regression-v1', endpoint='https://huggingface.co', repo_type='model', repo_id='capston-team-5/finma-7b-lora-regression-v1'), pr_revision=None, pr_num=None)